In [1]:
# import scraping dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# set up splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# define and visit url
url = 'https://www.footballoutsiders.com/stats/nfl/pace-stats/2022?check_logged_in=1'
browser.visit(url)

In [4]:
# define user info
username = 'davidgornowicz@gmail.com'
password = 'teampassword'

In [5]:
# find and fill username
username_box = browser.find_by_id('edit-name')
username_box.fill(username)

In [6]:
# find and fill password
password_box = browser.find_by_id('edit-pass')
password_box.fill(password)

In [7]:
# find and click login button
submit = browser.find_by_id('edit-submit').click()

In [8]:
# not working plan b
dwnld_csv_bttn = browser.find_by_text('new-table-csv-download-image').click()

ElementDoesNotExist: no elements could be found with text "new-table-csv-download-image"

In [9]:
# download the html to scrape
html = browser.html
html_soup = soup(html, 'html.parser')

In [10]:
# select the table
offense_table = html_soup.find_all('table')[0]
offense_table

<table class="new-table responsive-enabled" data-once="tableresponsive" data-striping="1" style="margin-left: -42px;">
<thead>
<tr>
<th>Team</th>
<th colspan="2">Sec/Play<br/>(total)</th>
<th colspan="2">Sec/Play<br/>(1st half)</th>
<th colspan="2">Sec/Play<br/>(2nd half)</th>
<th colspan="2">Sec/Play<br/>(lead 7+)</th>
<th colspan="2">Sec/Play<br/>(+/- 6)</th>
<th colspan="2">Sec/Play<br/>(trail 7+)</th>
<th colspan="2">Sec/Play<br/>(situation<br/>neutral)</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td><a href="/team/TB">TB</a></td>
<td class="new-table-left-joined">1</td>
<td class="new-table-right-joined">24.93</td>
<td class="new-table-left-joined">3</td>
<td class="new-table-right-joined">26.28</td>
<td class="new-table-left-joined">1</td>
<td class="new-table-right-joined">23.66</td>
<td class="new-table-left-joined">3</td>
<td class="new-table-right-joined">22.56</td>
<td class="new-table-left-joined">16</td>
<td class="new-table-right-joined">29.00</td>
<td class="new-table-l

In [11]:
# EXPERIMENTAL seperating td into own list
offense_table_list = offense_table.find_all(['td'], class_='new-table-right-joined')
offense_table_list

[<td class="new-table-right-joined">24.93</td>,
 <td class="new-table-right-joined">26.28</td>,
 <td class="new-table-right-joined">23.66</td>,
 <td class="new-table-right-joined">22.56</td>,
 <td class="new-table-right-joined">29.00</td>,
 <td class="new-table-right-joined">21.07</td>,
 <td class="new-table-right-joined">29.39</td>,
 <td class="new-table-right-joined">25.90</td>,
 <td class="new-table-right-joined">26.32</td>,
 <td class="new-table-right-joined">25.51</td>,
 <td class="new-table-right-joined">28.33</td>,
 <td class="new-table-right-joined">27.40</td>,
 <td class="new-table-right-joined">22.30</td>,
 <td class="new-table-right-joined">28.68</td>,
 <td class="new-table-right-joined">26.19</td>,
 <td class="new-table-right-joined">27.23</td>,
 <td class="new-table-right-joined">25.20</td>,
 <td class="new-table-right-joined">24.68</td>,
 <td class="new-table-right-joined">28.77</td>,
 <td class="new-table-right-joined">23.05</td>,
 <td class="new-table-right-joined">30.4

In [12]:
# find headers
headers = []
for i in offense_table.find_all('th'):
    title = i.text
    headers.append(title)
headers

['Team',
 'Sec/Play(total)',
 'Sec/Play(1st half)',
 'Sec/Play(2nd half)',
 'Sec/Play(lead 7+)',
 'Sec/Play(+/- 6)',
 'Sec/Play(trail 7+)',
 'Sec/Play(situationneutral)']

In [13]:
# create df for offensive table
df_offense_pace = pd.DataFrame(columns = headers)
df_offense_pace

,Team,Sec/Play(total),Sec/Play(1st half),Sec/Play(2nd half),Sec/Play(lead 7+),Sec/Play(+/- 6),Sec/Play(trail 7+),Sec/Play(situationneutral)


In [14]:
# add rows to df (no class specifier, runs but with wrong columns)
for j in offense_table.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [i.text for i in row_data]
    df_offense_pace.append(pd.Series(row, index=df_offense_pace.columns[:len(row)]), ignore_index=True)    
df_offense_pace

ValueError: Length of values (15) does not match length of index (8)

In [15]:
# add rows to df (class specifier returns no data)
for j in offense_table.find_all('tr')[1:]:
    row_data = j.find_all('td', class_='new-table-right-joined')
    row = [i.text for i in row_data]
    df_offense_pace.append(pd.Series(row, index=df_offense_pace.columns[:len(row)]), ignore_index=True)    
df_offense_pace

,Team,Sec/Play(total),Sec/Play(1st half),Sec/Play(2nd half),Sec/Play(lead 7+),Sec/Play(+/- 6),Sec/Play(trail 7+),Sec/Play(situationneutral)
